In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

In [5]:
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)


0.4%

100.0%


Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100.0%


Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



100.0%


Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100.0%

Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



In [6]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=4, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=4, shuffle=False)

In [7]:
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

In [8]:
# Report split sizes
print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(validation_set)))

Training set has 60000 instances
Validation set has 10000 instances


In [9]:
from models import CNN_Simple
model = CNN_Simple()

In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [12]:
loss_fn = torch.nn.CrossEntropyLoss()

In [14]:
def train_one_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.

    return last_loss

In [15]:
# Initializing in a separate cell so we can easily add more epochs to the same run
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number)


    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    epoch_number += 1

EPOCH 1:
  batch 1000 loss: 1.4475891344156115
  batch 2000 loss: 0.7376667435467243
  batch 3000 loss: 0.663162180387415
  batch 4000 loss: 0.5893003825994674
  batch 5000 loss: 0.578485480874544
  batch 6000 loss: 0.5154063177620992
  batch 7000 loss: 0.4999772820346989
  batch 8000 loss: 0.5021630440687295
  batch 9000 loss: 0.4896695383652113
  batch 10000 loss: 0.4522196855896618
  batch 11000 loss: 0.45328562531716304
  batch 12000 loss: 0.4376799937356263
  batch 13000 loss: 0.4270633884431445
  batch 14000 loss: 0.4049722499338677
  batch 15000 loss: 0.40121850893145894
LOSS train 0.40121850893145894 valid 0.41795220971107483
EPOCH 2:
  batch 1000 loss: 0.40466134119685737
  batch 2000 loss: 0.37939334598983987
  batch 3000 loss: 0.4128810850041336
  batch 4000 loss: 0.37702266152389347
  batch 5000 loss: 0.3666479909853806
  batch 6000 loss: 0.3873753381916031
  batch 7000 loss: 0.37400172716213276
  batch 8000 loss: 0.3727881022139918
  batch 9000 loss: 0.36785975900909396
  